In [ ]:
'''
Importa os módulos usados
'''
import numpy as np
import matplotlib.pyplot as plt
import copy
import pandas as pd

class Struct:
    pass

def wrap_struct(solution):
    x = Struct()
    x.solution = solution
    return x

# Leitura dos dados
path_equip = 'Dados/EquipDB.csv'
path_planos = 'Dados/MPDB.csv'
path_cluster = 'Dados/ClusterDB.csv'

Equipamentos = pd.read_csv(path_equip, header=None, names=["ID", "t0", "cluster", "custo de falha"])
Planos = pd.read_csv(path_planos, header=None, names=["ID", "k", "custo"])
ModeloFalha = pd.read_csv(path_cluster, header=None, names=["ID", "eta", "beta"])

# Solução inicial aleatória
def Sol_Inicial(equipamentos):
    return np.random.randint(1, 4, size=len(equipamentos))

# Estrutura de dados
def manutencao_def(equipamentos, planos):
    dados = Struct()
    dados.equipamentos = equipamentos
    dados.planos = planos.set_index("ID")
    dados.modelo_falha = ModeloFalha.set_index("ID")
    dados.n = len(equipamentos)
    dados.custo_por_plano = dados.planos["custo"].to_dict()
    return dados

# Funções objetivo
def fobj_f1(x, dados):
    custo_total = sum(dados.custo_por_plano[plano] for plano in x.solution)
    x.fitness = custo_total
    return x

def fobj_f2(x, dados):
    custo = 0
    for i, plano in enumerate(x.solution):
        eq = dados.equipamentos.iloc[i]
        cluster = dados.modelo_falha.loc[eq["cluster"]]
        plano_info = dados.planos.loc[plano]

        t0 = eq["t0"]
        k = plano_info["k"]
        delta_t = 5
        eta = cluster["eta"]
        beta = cluster["beta"]

        Fi = lambda t: 1 - np.exp(-(t / eta)**beta)
        pi = (Fi(t0 + k * delta_t) - Fi(t0)) / (1 - Fi(t0))
        custo += pi * eq["custo de falha"]

    x.fitness = custo
    return x

# Soma ponderada
def soma_ponderada(x, dados, peso_f1):
    peso_f2 = 100 - peso_f1
    f1 = fobj_f1(copy.deepcopy(x), dados).fitness
    f2 = fobj_f2(copy.deepcopy(x), dados).fitness
    
    # Normalização (ajuste os valores de referência conforme necessário)
    ref_f1 = 1049  # Referência para f1 (custo total de manutenção)
    ref_f2 = 1744  # Referência para f2 (custo total de falha)
    
    f1_norm = f1 / ref_f1
    f2_norm = f2 / ref_f2
    
    x.fitness = (f1_norm * peso_f1 + f2_norm * peso_f2) / 100
    return x

# Operadores de vizinhança
def shake(x, r, planos=3):
    y = copy.deepcopy(x)
    indices = np.random.permutation(len(x.solution))[:r]
    for idx in indices:
        plano_atual = y.solution[idx]
        opcoes = [p for p in range(1, planos+1) if p != plano_atual]
        y.solution[idx] = np.random.choice(opcoes)
    return y

def shake_adiciona(x, r):
    y = copy.deepcopy(x)
    indices = np.random.permutation(len(y.solution))[:r]
    for idx in indices:
        if y.solution[idx] < 3:
            y.solution[idx] += 1
    return y

def shake_subtrai(x, r):
    y = copy.deepcopy(x)
    indices = np.random.permutation(len(y.solution))[:r]
    for idx in indices:
        if y.solution[idx] > 1:
            y.solution[idx] -= 1
    return y

# Algoritmo RVNS otimizado
def rvns_otimizado(fobj, sol_inicial, shake, max_num_sol_avaliadas, r=10, kmax=3):
    x = wrap_struct(sol_inicial())
    x = fobj(x)
    
    num_sol_avaliadas = 1
    historico = [x.fitness]
    best = copy.deepcopy(x)
    historico_best = [best.fitness]

    estruturas_vizinhanca = [shake_adiciona, shake_subtrai, shake]

    while num_sol_avaliadas < max_num_sol_avaliadas:
        k = 1
        while k <= kmax and num_sol_avaliadas < max_num_sol_avaliadas:
            for shake_func in estruturas_vizinhanca:
                if num_sol_avaliadas >= max_num_sol_avaliadas:
                    break
                    
                y = shake_func(x, r)
                y = fobj(y)
                num_sol_avaliadas += 1
                historico.append(x.fitness)
                
                if y.fitness < best.fitness:
                    best = copy.deepcopy(y)
                historico_best.append(best.fitness)

                if y.fitness < x.fitness:
                    x = copy.deepcopy(y)
                    break
            else:
                k += 1

    return best, historico

# Gera a fronteira de Pareto
def gerar_fronteira_pareto(dados, num_pontos=20, max_iteracoes=100):
    pesos = np.linspace(0, 100, num_pontos)
    resultados_f1 = []
    resultados_f2 = []
    
    for peso in pesos:
        print(f"Otimizando para peso f1 = {peso:.1f}, f2 = {100 - peso:.1f}")
        
        def fobj_personalizada(x):
            return soma_ponderada(x, dados, peso)
        
        best, _ = rvns_otimizado(
            fobj=fobj_personalizada,
            sol_inicial=lambda: Sol_Inicial(Equipamentos),
            shake=shake,
            max_num_sol_avaliadas=max_iteracoes,
            r=10,
            kmax=3
        )
        
        f1 = fobj_f1(copy.deepcopy(best), dados).fitness
        f2 = fobj_f2(copy.deepcopy(best), dados).fitness
        
        resultados_f1.append(f1)
        resultados_f2.append(f2)
    
    return resultados_f1, resultados_f2

# Execução principal
if __name__ == "__main__":
    dados = manutencao_def(Equipamentos, Planos)
    
    print("\n--- GERANDO FRONTEIRA DE PARETO ---\n")
    resultados_f1, resultados_f2 = gerar_fronteira_pareto(dados, num_pontos=200, max_iteracoes=50)
    
    # Gráfico da fronteira de Pareto
    plt.figure(figsize=(10, 6))
    plt.scatter(resultados_f2, resultados_f1, c='blue', label='Soluções')
    plt.title("Fronteira de Pareto Aproximada")
    plt.xlabel("Custo de Falha (F2)")
    plt.ylabel("Custo de Manutenção (F1)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


--- GERANDO FRONTEIRA DE PARETO ---

Otimizando para peso f1 = 0.0, f2 = 100.0
Otimizando para peso f1 = 1.0, f2 = 99.0
Otimizando para peso f1 = 2.0, f2 = 98.0
Otimizando para peso f1 = 3.0, f2 = 97.0
Otimizando para peso f1 = 4.0, f2 = 96.0


KeyboardInterrupt: 